### Libraries

In [2]:
import pandas as pd
import numpy as np 

from pyspark.sql import SparkSession
from pyspark.sql import functions as F
from pyspark.sql.types import *
import os
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
#Spark version and Spark NLP version should be align

### Data Understanding

In [66]:
spark = SparkSession.builder\
    .appName("nlp")\
    .master("local[*]")\
    .config("spark.driver.memory","8G")\
    .config("spark.driver.maxResultSize", "0") \
    .config("spark.kryoserializer.buffer.max", "2000M")\
    .getOrCreate()

spark.sparkContext.setLogLevel("ERROR")

In [67]:
data_path = os.environ.get("NEWS_DATA_PATH")

In [68]:
!head -c 300 $data_path

,Unnamed: 0,date,year,month,day,author,title,article,url,section,publication
0,0,2016-12-09 18:31:00,2016,12.0,9,Lee Drutman,We should take concerns about the health of liberal democracy seriously,"This post is part of Polyarchy, an independent blog produced by the political reform program at New Am

In [69]:
!tail -c 300 $data_path

comes to fitness. On another note ... it sounds like she's still working when it comes to SCOTUS biz too. Law360 notes she calls into conference chats via phone with the other robes these days.",https://www.tmz.com/2020/04/01/ruth-bader-ginsburg-rbg-justice-working-out-gym-trainer-coronavirus/,,TMZ


The columns have to be change for a more readable names

In [81]:
df = spark.read.option("header","true").csv(data_path).withColumnRenamed("_c0","idx").withColumnRenamed("Unnamed: 0","idx_news")

In [82]:
df.printSchema()

root
 |-- idx: string (nullable = true)
 |-- idx_news: string (nullable = true)
 |-- date: string (nullable = true)
 |-- year: string (nullable = true)
 |-- month: string (nullable = true)
 |-- day: string (nullable = true)
 |-- author: string (nullable = true)
 |-- title: string (nullable = true)
 |-- article: string (nullable = true)
 |-- url: string (nullable = true)
 |-- section: string (nullable = true)
 |-- publication: string (nullable = true)



In [83]:
df.show(3,vertical=True,truncate=150)

-RECORD 0-------------------------------------------------------------------------------------------------------------------------------------------------------------
 idx         | 0                                                                                                                                                      
 idx_news    | 0                                                                                                                                                      
 date        | 2016-12-09 18:31:00                                                                                                                                    
 year        | 2016                                                                                                                                                   
 month       | 12.0                                                                                                                                                  

In [84]:
df.count()

3858240

In [76]:
import pyspark.sql.functions as F

In [80]:
df.groupBy("year").count().show()

+--------------------+-----+
|                year|count|
+--------------------+-----+
| ""Atlas Obscura"...|    1|
| like most Androi...|    1|
| too. In ""Call o...|    1|
| Brinkley remains...|    1|
| Will's survival ...|    1|
| but still a heal...|    1|
| meant I was not ...|    1|
|               beans|    6|
| the doors are armed|    1|
| the young spoile...|    2|
| lots of people s...|    1|
| 74% are in favor...|    1|
| but extends into...|    1|
| follow him on In...|    1|
| your skin will p...|    1|
| a counselor and ...|    1|
|    once and for all|    1|
| this cool applia...|    2|
|                $39 |    1|
| "" Zuckerberg says.|    1|
+--------------------+-----+
only showing top 20 rows

